# Cython Tutorial

This is a topical tutorial. If you are new to iminuit, you should go through the basic tutorial first. 

If you do not already know Cython and you just want to make your code faster, try **numba** or **jax** to accelerate your cost function (see the automatic differentiation tutorial for more details). These are easier to use and more powerful, and you don't have to learn the awkward Cython dialect. iminuit also works with PyPy3.5 or later, by the way.

Apart from showing how to use Cython functions with iminuit, this short tutorial focusses on workarounds that are needed when iminuit's introspection cannot detect the parameter names automatically.

In [1]:
# setup of the notebook
%load_ext Cython
from iminuit import Minuit, describe
from iminuit.util import make_func_code
import numpy as np
import traceback

Most of the computation time is spend in the cost function when you run iminuit. Cython is (an old-fashioned) way to accelerate it. We will only give a very brief glimps on how to write Cython code, please search the web for a proper Cython tutorial. The following cell is Cython code and will be compiled to machine code behind the scenes.

In [2]:
%%cython

def cython_func(double x, double y, double z):
    return (x - 1.) ** 2 + (y - 2.) ** 2 + (z - 3.) ** 2 + 1.

Unfortunately, if we pass this to iminuit, we get a failure.

In [3]:
try:
    m = Minuit(cython_func, pedantic=False)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-3-faebe1f5e16a>", line 2, in <module>
    m = Minuit(cython_func, pedantic=False)
  File "src/iminuit/_libiminuit.pyx", line 587, in iminuit._libiminuit.Minuit.__init__
  File "/Users/hdembinski/Extern/iminuit/src/iminuit/util.py", line 404, in describe
    raise TypeError("Unable to obtain function signature")
TypeError: Unable to obtain function signature


What happened? `Minuit` uses the `describe` tool which uses introspection to read the function signature, but this failed here. Without that, Minuit does not know how many parameters this function accepts and their names.

Python built-in functions (like `min`) normally do not have a function signature. Functions from cython and swig also do not have one.

As a simple fix, you can pass the parameter names to Minuit, then it works.

In [4]:
m = Minuit(cython_func, name=("x", "y", "z"), pedantic=False)
m.migrad()

------------------------------------------------------------------
| FCN = 1                       |      Ncalls=36 (36 total)      |
| EDM = 2.26e-19 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x    |     1     |     1     |            |            |         |         |       |
| 1 | y    |     2     |     1     |            |            |         |         |       |
| 2 | z    |     3     |     1     |            |            |         |         |       |
------------------------------------------------------------------------------------------

A nicer solution is to ask Cython to add the missing function signature. This can be achieved with the `embedsignature(true)` decorator.

In [5]:
%%cython
cimport cython

@cython.embedsignature(True)  # generate a signature that iminuit can extact
def cython_f(double x, double y, double z):
    return (x - 1.) ** 2 + (y - 2.) ** 2 + (z - 3.) ** 2 + 1.

In [6]:
m = Minuit(cython_f, pedantic=False, errordef=1)
m.migrad()

------------------------------------------------------------------
| FCN = 1                       |      Ncalls=36 (36 total)      |
| EDM = 2.26e-19 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x    |     1     |     1     |            |            |         |         |       |
| 1 | y    |     2     |     1     |            |            |         |         |       |
| 2 | z    |     3     |     1     |            |            |         |         |       |
------------------------------------------------------------------------------------------

Now it works without forcing parameter names.